In [25]:
import pandas as pd
from collections import Counter
import nltk
import re
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

keywordsJson = pd.read_json('../data/depression_synonyms.json', orient='records')
keywords = list(keywordsJson['depression'])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bramb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Create vectors

In [44]:
df = pd.read_json('../data/splitData/postdataLinesSplit.json',lines=True)

def preprocess(post):
    preprocessedWords = []
    lemmatizer = WordNetLemmatizer()
    post = post[0]
    post = post.lower()
    post = re.sub(r'[^\w\s]', '', post)
    words = post.split()
    for word in words:
        lemmatized = lemmatizer.lemmatize(word)
        preprocessedWords.append(lemmatized)
    
    return preprocessedWords

prediction = {'prediction':[]}

for index, row in df.iterrows():
    DepressionWordCount = 0
    text = row['text']
    preProcessed = preprocess(text)
    wordCounts = dict(Counter(preProcessed))
    keywordCounts = {}
    keywordCounts['id'] = index
    for word in keywords:
        try:
            DepressionWordCount += wordCounts[word]
        except: 
            continue
    if DepressionWordCount >= 2:
        prediction['prediction'].append('depression')
    else:
        prediction['prediction'].append('normal')
    

vector_df = pd.DataFrame(prediction)
df = pd.concat([df, vector_df], axis=1)
df.to_json("../data/predictionData/ruleBasedPred.json", orient='records', lines=True)